Process model for constant velocity model from euler angles.

Using euler angle convention of zyx, (yaw, pitch, roll)

In [1]:
import numpy as np
import sympy
import symforce.symbolic as sf

In [4]:
def R_z(angle : float) -> np.array:
    R_z = np.array([[np.cos(angle), -np.sin(angle), 0],
                    [np.sin(angle),  np.cos(angle), 0],
                    [0, 0, 1]])
    return R_z

def R_y(angle: float)-> np.array :
    R_y = np.array([[np.cos(angle), 0, np.sin(angle)],
                    [0, 1, 0],
                    [-np.sin(angle), 0, np.cos(angle)]])
    return R_y

def R_x(angle: float) -> np.array:
    R_x = np.array([[1, 0 , 0],
                    [0, np.cos(angle), -np.sin(angle)],
                    [0, np.sin(angle), np.cos(angle)]])
    return R_x

def euler_angle_to_rot(yaw: float, pitch: float, roll: float):
    
    return R_z(yaw) @ R_y(pitch) @ R_x(roll)

# http://www.stengel.mycpanel.princeton.edu/Quaternions.pdf
def euler_angle_rates_to_ang_vel(yaw_rate:float, pitch_rate:float, roll_rate:float, yaw:float, pitch:float, roll:float) -> np.array:
    """
    converts euler angles rates to body axis rates. 
    """
    euler_angle_rate_vec = np.array([roll_rate, pitch_rate, yaw_rate]).reshape(3, -1)
    psi = yaw
    theta = pitch
    phi = roll
    rate_matrix = np.array([
        [1, 0, -np.sin(theta)],
        [0, np.cos(phi), np.cos(theta)*np.sin(phi)],
        [0, -np.sin(phi), np.cos(phi)*np.cos(theta)]
    ])
    
    ang_vel = rate_matrix @ euler_angle_rate_vec
    
    return ang_vel


def ang_vel_to_euler_angle_rates(omega : np.array, yaw: float, pitch:float, roll:float):
    """
    omega - body axis rates
    
    """
    psi = yaw
    theta = pitch
    phi = roll
    omega = omega.reshape(3, -1)
    rate_matrix = np.array([
        [1, np.sin(phi)*np.tan(theta), np.cos(phi)*np.tan(theta)],
        [0, np.cos(phi), -np.sin(phi)],
        [0, np.sin(phi)*np.sec(theta), np.cos(phi)*np.sec(theta)]
    ])
    euler_angle_rates = rate_matrix @ omega
    
    return euler_angle_rates

def orientation_error_propagation(yaw:float, pitch:float, roll:float, body_omega:np.array):
    wx, wy, wz = body_omega
    qx = roll
    qy = pitch
    qz = yaw
    cx = np.cos(qx)
    sx = np.sin(qx)
    ty = np.tan(qy)
    sy = np.sin(qy)
    cy = np.cos(qy)
    
    
    A = np.array([
        [0, (wz*cx + wy*sx)/cy^2, ty*(wy*cx - wz*sx)],
        [0, 0, wz*cx + wy*sx],
        [0, (sy*(wz*cx + wy*sx))/cy^2, (wy * cx - wz*sx)/cy]
    ])
    
    return A@body_omega.reshape(3, -1)

Alternatives to euler angles  is rotation matrix (or direction cosines) and quaternions to avoid singularities.


In [ ]:
def skew(vec: np.array):
    M = np.array([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0],
    ])
    
    return M
    
def vee(vec: np.array):
    M = np.array([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0],
    ])
    return M


def angular_vel_to_rotation_rates(R_curr: np.array, omega: np.array):
    """
    angular error 
    """
    R_dot = R_curr@skew(omega)
    
    return R_dot    